In [1]:
import sys
import os
import time
import sqlite3
import math
from pathlib import Path
from itertools import chain
from collections import Counter
from typing import Callable

import numpy as np
import pandas as pd
import scipy as sp
from sklearn.cluster import DBSCAN
from ordered_set import OrderedSet as oset

sys.path.append("..")
from filesplitter import db, subjects, naming
from filesplitter.clustering import cluster_dataset, to_name_cluster_labels
from filesplitter.loading import load_dataset
from filesplitter.validate import validate
from filesplitter.graph import group_by_scc, group_by_wcc, group_edges_by
from filesplitter.naming import NameSimilarity

In [2]:
# DS = subjects.load_subject(subjects.ANDROID_BASE_TEXT_VIEW)

In [ ]:
DS = load_dataset("../data/android-base.db", "services/core/java/com/android/server/audio/AudioService.java")

In [3]:
DS.targets_df

,parent_id,name,kind,start_row,end_row
id,,,,,
492564,1323,LOG_TAG,field,368,368
492565,1323,DEBUG_EXTRACT,field,369,369
492566,1323,DEBUG_CURSOR,field,370,370
492567,1323,TEMP_POSITION,field,372,372
492568,1323,XMLTypefaceAttr,annotation,377,379
...,...,...,...,...,...
289503,1323,onInputConnectionOpenedInternal,method,14211,14218
289505,1323,onInputConnectionClosedInternal,method,14221,14226
246325,1323,onReceiveContent,method,14245,14252


In [4]:
entities_df = cluster_dataset(DS)

[W0]               (19:58:29)   Starting... (5574 edges and 1231 nodes = 4.5280 density)	Bisected with a cut weight of 568595.0 in 32.3546 secs.
[W0A]              (19:59:02)   Starting... (4509 edges and 1053 nodes = 4.2821 density)	Bisected with a cut weight of 710783.0 in 41.9715 secs.
[W0AA]             (19:59:44)   Starting... (412 edges and 180 nodes = 2.2889 density)	Bisected with a cut weight of 143147.0 in 4.0424 secs.
[W0AAA]            (19:59:48)   Starting... (37 edges and 46 nodes = 0.8043 density)	Bisected with a cut weight of 15813.0 in 0.7825 secs.
[W0AAAA]           (19:59:49)   Starting... (5 edges and 7 nodes = 0.7143 density)	Aborted. Weight under threshold.
[W0AAAB]           (19:59:49)   Starting... (32 edges and 39 nodes = 0.8205 density)	Aborted. Weight under threshold.
[W0AAB]            (19:59:49)   Starting... (327 edges and 134 nodes = 2.4403 density)	Bisected with a cut weight of 185320.0 in 6.5287 secs.
[W0AABA]           (19:59:55)   Starting... (233 edge

In [5]:
entities_df

,parent_id,name,kind,start_row,end_row,name_id,strong_id,weak_id,block_name,block_id
id,,,,,,,,,,
492564,1323,LOG_TAG,field,368.0,368.0,53,0,0,W0ABBAB,35
492565,1323,DEBUG_EXTRACT,field,369.0,369.0,13,1,0,W0AABABBBB,7
492566,1323,DEBUG_CURSOR,field,370.0,370.0,12,2,0,W0AABABBBB,7
492567,1323,TEMP_POSITION,field,372.0,372.0,71,3,0,W0BABAA,44
492568,1323,XMLTypefaceAttr,annotation,377.0,379.0,78,4,1,W1,51
...,...,...,...,...,...,...,...,...,...,...
815562,None,tests/VoiceInteraction/src/com/android/test/vo...,file,NaN,NaN,1290,1296,0,W0AAAB,1
815601,None,tests/VoiceInteraction/src/com/android/test/vo...,file,NaN,NaN,1291,1297,0,W0AAAB,1
815652,None,tests/WallpaperTest/src/com/example/wallpapert...,file,NaN,NaN,1292,1298,0,W0ABAAABBA,17


In [6]:
entities_df[entities_df["block_id"] == 0]

,parent_id,name,kind,start_row,end_row,name_id,strong_id,weak_id,block_name,block_id
id,,,,,,,,,,
492911,1323,setTextKeepState,method,6423.0,6426.0,1146,448,0,W0AAAA,0
492912,1323,setTextKeepState,method,6720.0,6734.0,1146,448,0,W0AAAA,0
492971,1323,hasOverlappingRendering,method,8073.0,8079.0,549,487,0,W0AAAA,0
493062,1323,onGenericMotionEvent,method,11530.0,11544.0,764,589,0,W0AAAA,0
493069,1323,onTrackballEvent,method,11594.0,11603.0,794,594,0,W0AAAA,0
493071,1323,getLeftFadingEdgeStrength,method,11614.0,11629.0,465,597,0,W0AAAA,0
493072,1323,getRightFadingEdgeStrength,method,11631.0,11644.0,494,598,0,W0AAAA,0
493073,1323,getHorizontalFadingEdgeStrength,method,11655.0,11662.0,445,596,0,W0AAAA,0
